# RNN "Many-2-Many" use-case. 

## Run sequences-sequences based on char (char in word, or subword).

### Dataset layout

#### Window 0:

```python
Sample: "Hello,world"
Label: "ello,world"
```

#### Window 1:

```python
Sample: "ello,world"
Label: "llo,world"
```
....

In [1956]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRU, Conv1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 
import pprint as pp

In [1957]:
tokenizer = Tokenizer(split='\n', char_level=False) # Subword-2-Subword, char-2-char, but we keep using word based tokenizer for using filters further.

data= "In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."
 
print(data)


In the town of Athy one Jeremy Lanigan 
 Battered away til he hadnt a pound. 
His father died and made him a man again 
 Left him a farm and ten acres of ground. 
He gave a grand party for friends and relations 
Who didnt forget him when come to the wall, 
And if youll but listen Ill make your eyes glisten 
Of the rows and the ructions of Lanigans Ball. 
Myself to be sure got free invitation, 
For all the nice girls and boys I might ask, 
And just in a minute both friends and relations 
Were dancing round merry as bees round a cask. 
Judy ODaly, that nice little milliner, 
She tipped me a wink for to give her a call, 
And I soon arrived with Peggy McGilligan 
Just in time for Lanigans Ball. 
There were lashings of punch and wine for the ladies, 
Potatoes and cakes; there was bacon and tea, 
There were the Nolans, Dolans, OGradys 
Courting the girls and dancing away. 
Songs they went round as plenty as water, 
The harp that once sounded in Taras old hall,
Sweet Nelly Gray and The Rat Ca

In [1958]:

tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1
print(total_words)
print(tokenizer.word_index)


27
{' ': 1, 'e': 2, 'a': 3, 'n': 4, 't': 5, 'i': 6, 'l': 7, 's': 8, 'r': 9, 'o': 10, 'h': 11, 'd': 12, 'g': 13, 'p': 14, 'u': 15, 'c': 16, 'w': 17, 'm': 18, 'f': 19, 'y': 20, 'b': 21, 'k': 22, 'j': 23, 'v': 24, 'q': 25, 'z': 26}


In [1959]:
print(tokenizer.word_index['i'])
print(tokenizer.word_index['t'])
print(tokenizer.word_index['w']) 


6
5
17


In [1960]:
def flat_list(l): return sum(l, [])

In [1961]:
print(data[:10])

input_sequences = tokenizer.texts_to_sequences(data)
print(input_sequences[:10]) 
# texts_to_sequences returns a list X list.
input_sequences = flat_list(input_sequences) 
print(input_sequences[:10])

In the tow
[[6], [4], [1], [5], [11], [2], [1], [5], [10], [17]]
[6, 4, 1, 5, 11, 2, 1, 5, 10, 17]


In [1962]:
window_size = 11
batch_size = 1
 
input_sequences = np.expand_dims(input_sequences, axis=-1)
print(input_sequences[:10])

dataset = tf.data.Dataset.from_tensor_slices(input_sequences)
dataset = dataset.window(window_size, 1, 1, True)
dataset = dataset.flat_map(lambda window: window.batch(window_size).map(lambda x: (x[:-1], x[1:])))
dataset = dataset.batch(batch_size, drop_remainder=True)

for item in dataset.take(2):
  print(item)
  print()


[[ 6]
 [ 4]
 [ 1]
 [ 5]
 [11]
 [ 2]
 [ 1]
 [ 5]
 [10]
 [17]]
(<tf.Tensor: shape=(1, 10, 1), dtype=int64, numpy=
array([[[ 6],
        [ 4],
        [ 1],
        [ 5],
        [11],
        [ 2],
        [ 1],
        [ 5],
        [10],
        [17]]])>, <tf.Tensor: shape=(1, 10, 1), dtype=int64, numpy=
array([[[ 4],
        [ 1],
        [ 5],
        [11],
        [ 2],
        [ 1],
        [ 5],
        [10],
        [17],
        [ 4]]])>)

(<tf.Tensor: shape=(1, 10, 1), dtype=int64, numpy=
array([[[ 4],
        [ 1],
        [ 5],
        [11],
        [ 2],
        [ 1],
        [ 5],
        [10],
        [17],
        [ 4]]])>, <tf.Tensor: shape=(1, 10, 1), dtype=int64, numpy=
array([[[ 1],
        [ 5],
        [11],
        [ 2],
        [ 1],
        [ 5],
        [10],
        [17],
        [ 4],
        [ 1]]])>)



In [1963]:
epochs = 150
units = 64
embeding_dim = 225 


In [1964]:
class StopCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    self.model.reset_states()
    #if float(logs.get('loss')) < 0.38:
      #print("\n[mae={}, loss={}]".format(logs.get('mae'), logs.get('loss')))
      #self.model.stop_training = True
stopCallback = StopCallback()


In [1965]:
def plot(history):
  import matplotlib.pyplot as plt
  
  acc = history.history['accuracy']
  loss = history.history['loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'b', label='Training accuracy')
  plt.title('Training accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training Loss')
  plt.title('Training loss')
  plt.legend()

  plt.show()
  

In [1966]:
def build_model(*layers):
  tf.keras.backend.clear_session()
  tf.random.set_seed(51)
  np.random.seed(51)

  model = Sequential()
  model.add(Embedding(total_words, embeding_dim, input_length=window_size - 1, batch_input_shape=[1, None]))
  [model.add(layer) for layer in layers]
  model.add(Dense(total_words / 2, 
                  activation=tf.keras.activations.relu,
                  kernel_regularizer=tf.keras.regularizers.l2(0.01)))
  model.add(Dense(total_words, activation=tf.keras.activations.softmax))

  # Use SparseCategoricalCrossentropy, cause we are not using one-hot based label
  # https://stats.stackexchange.com/questions/326065/cross-entropy-vs-sparse-cross-entropy-when-to-use-one-over-the-other
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
  print(model.summary())

  if len(layers) != 0:
    history = model.fit(dataset, epochs=epochs, verbose=1, callbacks=[stopCallback])

    weights = model.layers[0].get_weights()[0]
    print(weights.shape)

    plot(history)

  return model


In [1967]:
def predict_with_model(name, model, start_words="I", next_words=100):
  seed_text = start_words
    
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences(seed_text)  
    token_list = token_list = np.expand_dims(token_list, axis=0)

    if next_words == 1:
      print("token_list: {}".format(token_list))   

    predicted = model.predict(token_list)
    predicted_token_list = np.argmax(predicted, axis=-1)[0] 
    predicted_token_list = np.expand_dims(predicted_token_list, axis=1)

    if next_words == 1:
      print("predicted_token_list: {}".format(predicted_token_list))

    output_word = tokenizer.sequences_to_texts(predicted_token_list) 
    output_word = "".join(output_word)

    if next_words == 1:
      print("output_word: {}".format(output_word))

    seed_text += " " + output_word
  
  print("------------------------------")
  print(name)
  print(seed_text)  
  print("------------------------------")


In [1968]:
predict_with_model("Preview", 
                    build_model(), 
                    start_words="In the town",
                    next_words=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 225)            6075      
_________________________________________________________________
dense (Dense)                (1, None, 13)             2938      
_________________________________________________________________
dense_1 (Dense)              (1, None, 27)             378       
Total params: 9,391
Trainable params: 9,391
Non-trainable params: 0
_________________________________________________________________
None
token_list: [[[ 6]
  [ 4]
  [ 1]
  [ 5]
  [11]
  [ 2]
  [ 1]
  [ 5]
  [10]
  [17]
  [ 4]]]
predicted_token_list: [[17]
 [15]
 [ 4]
 [17]
 [ 4]
 [ 2]
 [ 4]
 [17]
 [20]
 [ 1]
 [15]]
output_word: wunwnenwy u
------------------------------
Preview
In the town wunwnenwy u
------------------------------


In [ ]:
model = build_model(LSTM(units, return_sequences=True, stateful=True))
predict_with_model("LSTM", 
                    model,
                    start_words="I went to Shanghai",
                    next_words=10)


In [ ]:
model = build_model(Bidirectional(LSTM(units, return_sequences=True, stateful=True)),
    LSTM(units, return_sequences=True, stateful=True))
predict_with_model("Bi-LSTM", 
                    model,                    
                    start_words="I went to Shanghai",
                    next_words=10)


In [ ]:
model = build_model(Bidirectional(LSTM(units, return_sequences=True, stateful=True)),
    Bidirectional(LSTM(units, return_sequences=True, stateful=True)),
    LSTM(units, return_sequences=True, stateful=True))
predict_with_model("2 X Bi-LSTM", 
                    model,                    
                    start_words="I went to Shanghai",
                    next_words=10)


In [ ]:
model = build_model(GRU(units, return_sequences=True, stateful=True))
predict_with_model("GRU", 
                    model,                    
                    start_words="I went to Shanghai",
                    next_words=10)


In [ ]:
model  = build_model(Bidirectional(GRU(units, return_sequences=True, stateful=True)),
    GRU(units, return_sequences=True, stateful=True))
predict_with_model("Bi-GRU", 
                    model,                    
                    start_words="I went to Shanghai",
                    next_words=10)


In [ ]:
model = build_model(Bidirectional(GRU(units, return_sequences=True, stateful=True)),
    Bidirectional(GRU(units, return_sequences=True, stateful=True)),
    GRU(units, return_sequences=True, stateful=True))
predict_with_model("2 X Bi-GRU", 
                    model,                    
                    start_words="I went to Shanghai",
                    next_words=10)
